In [1]:
import sys
import os
os.chdir("../")

In [2]:
import pandas as pd
import data
from tqdm.auto import tqdm
tqdm.pandas()
from IPython.display import display
pd.options.display.max_columns = None

In [3]:
train_df = data.train_df('small')

In [4]:
train_df.head(30)

,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices,frequence
0,00RL8Z82B2Z1,aff3928535f48,1541037460,1,search for poi,Newtown,AU,"Sydney, Australia",mobile,NaN,NaN,NaN,1
1,00RL8Z82B2Z1,aff3928535f48,1541037532,4,interaction item image,666856,AU,"Sydney, Australia",mobile,NaN,NaN,NaN,3
2,00RL8Z82B2Z1,aff3928535f48,1541037532,5,interaction item image,109038,AU,"Sydney, Australia",mobile,NaN,NaN,NaN,1
3,00RL8Z82B2Z1,aff3928535f48,1541037532,6,interaction item image,666856,AU,"Sydney, Australia",mobile,NaN,NaN,NaN,1
4,00RL8Z82B2Z1,aff3928535f48,1541037532,7,interaction item image,109038,AU,"Sydney, Australia",mobile,NaN,NaN,NaN,1
5,00RL8Z82B2Z1,aff3928535f48,1541037532,8,interaction item image,666856,AU,"Sydney, Australia",mobile,NaN,NaN,NaN,1
6,00RL8Z82B2Z1,aff3928535f48,1541037542,13,interaction item image,109038,AU,"Sydney, Australia",mobile,NaN,NaN,NaN,5
7,00RL8Z82B2Z1,aff3928535f48,1541037543,14,clickout item,109038,AU,"Sydney, Australia",mobile,NaN,3400638|1253714|3367857|5100540|1088584|666916...,95|66|501|112|95|100|101|72|82|56|56|143|70|25...,1
8,00RL8Z82B2Z1,aff3928535f48,1541038469,15,search for poi,Surry Hills,AU,"Sydney, Australia",mobile,NaN,NaN,NaN,1
9,00RL8Z82B2Z1,aff3928535f48,1541038485,16,clickout item,1257342,AU,"Sydney, Australia",mobile,NaN,55109|129343|54824|2297972|109014|1257342|1031...,162|25|150|143|101|49|118|131|18|100|101|143|5...,1


In [5]:
train_df = train_df.sort_index()

In [43]:
clickout_rows = train_df[['user_id','session_id','action_type','impressions','prices']][train_df.action_type == 'clickout item']
clickout_rows['impression_list'] = clickout_rows.impressions.str.split('|')
clickout_rows['price_list'] = clickout_rows.prices.str.split('|')

reference_rows = train_df[['user_id','session_id','reference','action_type']][train_df.reference.str.isnumeric() & (train_df.action_type != 'clickout item')].drop('action_type',axis=1)
reference_rows['price'] = 0

In [26]:
clickout_rows.head()

,user_id,session_id,action_type,impressions,prices,impression_list,price_list
7,00RL8Z82B2Z1,aff3928535f48,clickout item,3400638|1253714|3367857|5100540|1088584|666916...,95|66|501|112|95|100|101|72|82|56|56|143|70|25...,"[3400638, 1253714, 3367857, 5100540, 1088584, ...","[95, 66, 501, 112, 95, 100, 101, 72, 82, 56, 5..."
9,00RL8Z82B2Z1,aff3928535f48,clickout item,55109|129343|54824|2297972|109014|1257342|1031...,162|25|150|143|101|49|118|131|18|100|101|143|5...,"[55109, 129343, 54824, 2297972, 109014, 125734...","[162, 25, 150, 143, 101, 49, 118, 131, 18, 100..."
21,02SRUT1NQYH1,3599a6f709eab,clickout item,2795374|5582964|1088390|2781070|1258068|127196...,64|54|36|121|76|81|92|40|73|52|98|104|56|414|6...,"[2795374, 5582964, 1088390, 2781070, 1258068, ...","[64, 54, 36, 121, 76, 81, 92, 40, 73, 52, 98, ..."
26,03K8AXBL4BX2,ec139e10b9238,clickout item,12693|46363|81657|18448|47687|152913|18417|927...,104|92|100|103|102|104|72|85|81|75|107|86|98|8...,"[12693, 46363, 81657, 18448, 47687, 152913, 18...","[104, 92, 100, 103, 102, 104, 72, 85, 81, 75, ..."
43,03P4VFKK12UO,325fafb5fa450,clickout item,1306936|56482|2842358|6881276|65685|63259|6539...,178|104|110|94|57|96|46|61|48|35|50|38|59|44|5...,"[1306936, 56482, 2842358, 6881276, 65685, 6325...","[178, 104, 110, 94, 57, 96, 46, 61, 48, 35, 50..."


In [44]:
reference_rows.head(30)

,user_id,session_id,reference,price
1,00RL8Z82B2Z1,aff3928535f48,666856,0
2,00RL8Z82B2Z1,aff3928535f48,109038,0
3,00RL8Z82B2Z1,aff3928535f48,666856,0
4,00RL8Z82B2Z1,aff3928535f48,109038,0
5,00RL8Z82B2Z1,aff3928535f48,666856,0
6,00RL8Z82B2Z1,aff3928535f48,109038,0
10,02HGRBA06ODU,fa3a53156ae5c,3377332,0
11,02HGRBA06ODU,fa3a53156ae5c,3377332,0
12,02HGRBA06ODU,fa3a53156ae5c,3377332,0
13,02HGRBA06ODU,fa3a53156ae5c,3066244,0


In [45]:
#reference_rows['lista'] = 'cane'
#reference_rows['prezzi'] = 'gatto'

j = 0
clickout_indices = clickout_rows.index.values
for idx,row in tqdm(reference_rows.iterrows()):
    # find the next clickout index
    if idx >= clickout_indices[-1]:
        break
    while idx > clickout_indices[j]:
        j += 1
    next_clickout = clickout_rows.loc[clickout_indices[j]]

    # check if row and next_clickout are in the same session
    if row.user_id == next_clickout.user_id and row.session_id == next_clickout.session_id:
        try:
            ref_idx = next_clickout.impression_list.index(row.reference)
            reference_rows.at[idx,'price'] = next_clickout.price_list[ref_idx]
            
            #reference_rows.at[idx,'ref_indice'] = ref_idx
            #reference_rows.at[idx,'lista'] = ','.join(next_clickout.impression_list)
            #reference_rows.at[idx,'prezzi'] = ','.join(next_clickout.price_list)
        except:
            pass

In [46]:
reference_rows

,user_id,session_id,reference,price
1,00RL8Z82B2Z1,aff3928535f48,666856,56
2,00RL8Z82B2Z1,aff3928535f48,109038,56
3,00RL8Z82B2Z1,aff3928535f48,666856,56
4,00RL8Z82B2Z1,aff3928535f48,109038,56
5,00RL8Z82B2Z1,aff3928535f48,666856,56
6,00RL8Z82B2Z1,aff3928535f48,109038,56
10,02HGRBA06ODU,fa3a53156ae5c,3377332,0
11,02HGRBA06ODU,fa3a53156ae5c,3377332,0
12,02HGRBA06ODU,fa3a53156ae5c,3377332,0
13,02HGRBA06ODU,fa3a53156ae5c,3066244,0


In [47]:
reference_rows.dtypes

user_id       object
session_id    object
reference     object
price          int64
dtype: object

In [4]:
from extract_features.reference_price_in_next_clickout import ReferencePriceInNextClickout
f = ReferencePriceInNextClickout()

In [5]:
f.join_to(train_df)

/Users/federico/miniconda3/envs/recsys/lib/python3.6/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


reference_price_in_next_clickout feature read


,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices,frequence,price
0,00RL8Z82B2Z1,aff3928535f48,1541037460,1,search for poi,Newtown,AU,"Sydney, Australia",mobile,NaN,NaN,NaN,1,0
1,00RL8Z82B2Z1,aff3928535f48,1541037532,4,interaction item image,666856,AU,"Sydney, Australia",mobile,NaN,NaN,NaN,3,56
2,00RL8Z82B2Z1,aff3928535f48,1541037532,5,interaction item image,109038,AU,"Sydney, Australia",mobile,NaN,NaN,NaN,1,56
3,00RL8Z82B2Z1,aff3928535f48,1541037532,6,interaction item image,666856,AU,"Sydney, Australia",mobile,NaN,NaN,NaN,1,56
4,00RL8Z82B2Z1,aff3928535f48,1541037532,7,interaction item image,109038,AU,"Sydney, Australia",mobile,NaN,NaN,NaN,1,56
5,00RL8Z82B2Z1,aff3928535f48,1541037532,8,interaction item image,666856,AU,"Sydney, Australia",mobile,NaN,NaN,NaN,1,56
6,00RL8Z82B2Z1,aff3928535f48,1541037542,13,interaction item image,109038,AU,"Sydney, Australia",mobile,NaN,NaN,NaN,5,56
7,00RL8Z82B2Z1,aff3928535f48,1541037543,14,clickout item,109038,AU,"Sydney, Australia",mobile,NaN,3400638|1253714|3367857|5100540|1088584|666916...,95|66|501|112|95|100|101|72|82|56|56|143|70|25...,1,0
8,00RL8Z82B2Z1,aff3928535f48,1541038469,15,search for poi,Surry Hills,AU,"Sydney, Australia",mobile,NaN,NaN,NaN,1,0
9,00RL8Z82B2Z1,aff3928535f48,1541038485,16,clickout item,1257342,AU,"Sydney, Australia",mobile,NaN,55109|129343|54824|2297972|109014|1257342|1031...,162|25|150|143|101|49|118|131|18|100|101|143|5...,1,0
